In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys

print(os.getcwd())

/content


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 72 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 12.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595721 sha256=dfe6311b216f19570c559ccb1f2649568a3fc8fe9f354727121a3356159820d1
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 15.5 MB/s 
     |████████████████████████████████| 769 kB 14.9 MB/s 
     |████████████████████████████████| 3.0 MB 34.2 MB/s 
     |████████████████████████████████| 895 kB 67.9 MB/s 


In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-791zzreb
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-791zzreb
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12770 sha256=5c9944bbc984c1f06e7da857d6471f3932827e5ccb2fd584d725a555dc209609
  Stored in directory: /tmp/pip-ephem-wheel-cache-7aiyl862/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


# Data Preparation and Training

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [ ]:
import pandas as pd
chatbot_data = pd.read_csv('/content/drive/MyDrive/test/chatbot_sentiment_dataset.csv')

In [ ]:
chatbot_data.sample(n=10)

,Unnamed: 0,Sentence,Emotion
215838,183142,"갑자기라기보다, 처음부터 전제를 잘못 잡고 있었어. 걔는 학생회장을 하기 싫었던 게...",중립
154238,121542,하는 일 없이 노래교실에 다니는 친구와 내 처지가 비교 돼서 너무 화가 나.맞아. ...,분노
30735,36633,평화로운집회니 음악이어우러지니 어쩌니하니 닭은 진짜 축제같은건줄아나봄?,혐오
115229,82533,그가 나와의 결혼식을 며칠 앞두고 파혼을 요청해와서 상처받았어.응. 갑자기 파혼을 ...,슬픔
112504,79808,중년이 되니 부부 사이도 애틋한 거보단 각자 삶이 있는 것 같아.남편이 어느 순간 ...,당황
94601,61905,아들이 죽음에 대해서 질문하는데 내가 다 숙연해지더라.그렇지. 아들보다 내가 죽음에...,불안
206929,174233,포기한거 아냐.,중립
13592,19490,승진이빠르던가 예쁜여친이있던지 될사람은 되,슬픔
82886,50190,면접을 보러 갔는데 사무실에 간판이 없어서 혼란스러웠어.간판이 없는 것도 그렇고 면...,불안
190872,158176,의사선생한테 오늘도 연락 없어?,중립


In [ ]:
chatbot_data.loc[(chatbot_data['Emotion'] == "불안"), 'Emotion'] = 0  #불안 => 0
chatbot_data.loc[(chatbot_data['Emotion'] == "당황"), 'Emotion'] = 1  #당황 => 1
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['Emotion'] == "중립"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 5  #행복 => 5
chatbot_data.loc[(chatbot_data['Emotion'] == "혐오"), 'Emotion'] = 6  #혐오 => 6

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['우리는 미개 합니다', '2']
['지금 내모습은 컴퓨터앞에서 하루종일 취업사이트만 검색중...', '3']
['쉬는 시간에 오답이나 맞추고 있으면 모교로 돌아가서 중간고사 기말고사나 봐야지', '4']
[' 빨리빨리 파악해서 바로 고쳐주니깐 이러니 고객들이 믿고 살수밖에..', '5']
['미안한데 니들은 인간이 어떻게 이런말 쓰지마라', '6']
['그 여자랑 내가 무슨 상관인데? 아까는 탐정님이 부탁하기에 너 구하는 김에 주워왔지만, 민폐니까 얼른 나가.', '4']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.13, random_state=0)

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

190109
28408


In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
#Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [ ]:
# for data in train_dataloader:
#   print(data)
#   break

In [ ]:
#kobert 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
# train_dataloader

In [ ]:
best_acc=0.0
best_loss=99999999
ckpt_path="/content/drive/MyDrive/test/" #your own path
ckpt_name=ckpt_path+"saved_model.pt"

In [55]:
#kobert 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        # scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_loss=loss_fn(out,label)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {} test loss {}".format(e+1, test_acc / (batch_id+1),test_loss.data.cpu().numpy()))

    if test_acc>best_acc and test_loss.data.cpu().numpy()<best_loss:
      torch.save({'epoch':e+1,
                  'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'loss':test_loss.data.cpu().numpy()},
                 ckpt_name)
      best_loss=test_loss.data.cpu().numpy()
      bset_acc=test_acc
      
      print('current best model saved')
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


epoch 1 batch id 1 loss 0.16824722290039062 train acc 0.96875
epoch 1 batch id 201 loss 0.07003601640462875 train acc 0.9851523631840796
epoch 1 batch id 401 loss 0.0038964352570474148 train acc 0.9832839775561097
epoch 1 batch id 601 loss 0.10422869771718979 train acc 0.983153078202995
epoch 1 batch id 801 loss 0.01879844442009926 train acc 0.9830095193508115
epoch 1 batch id 1001 loss 0.026083270087838173 train acc 0.9826891858141859
epoch 1 batch id 1201 loss 0.048541221767663956 train acc 0.982826810990841
epoch 1 batch id 1401 loss 0.0507928729057312 train acc 0.9830032119914347
epoch 1 batch id 1601 loss 0.00401336932554841 train acc 0.9828915521549032
epoch 1 batch id 1801 loss 0.0845431387424469 train acc 0.9827005830094392
epoch 1 batch id 2001 loss 0.015039879828691483 train acc 0.9827351949025487
epoch 1 batch id 2201 loss 0.03284340724349022 train acc 0.9826073375738301
epoch 1 batch id 2401 loss 0.05721035227179527 train acc 0.9824357038733861
epoch 1 batch id 2601 loss 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



epoch 1 test acc 0.7747043918918919 test loss 1.5845586061477661
current best model saved


epoch 2 batch id 1 loss 0.13414990901947021 train acc 0.984375
epoch 2 batch id 201 loss 0.03162827715277672 train acc 0.9853855721393034
epoch 2 batch id 401 loss 0.002318520564585924 train acc 0.9831670822942643
epoch 2 batch id 601 loss 0.10684362798929214 train acc 0.9833610648918469
epoch 2 batch id 801 loss 0.06144106388092041 train acc 0.9830095193508115
epoch 2 batch id 1001 loss 0.010969359427690506 train acc 0.9830482017982018
epoch 2 batch id 1201 loss 0.0868084654211998 train acc 0.9831130308076603
epoch 2 batch id 1401 loss 0.11100742965936661 train acc 0.9833600999286224
epoch 2 batch id 1601 loss 0.005947912577539682 train acc 0.9832331355402874
epoch 2 batch id 1801 loss 0.022519104182720184 train acc 0.9828914491948917
epoch 2 batch id 2001 loss 0.006647337228059769 train acc 0.9831959020489756
epoch 2 batch id 2201 loss 0.05384117737412453 train acc 0.9832391526578828
epoch 2 batch id 2401 loss 0.042004507035017014 train acc 0.9830474281549354
epoch 2 batch id 2601 lo

...


epoch 8 test acc 0.775267454954955 test loss 1.8524469137191772


epoch 9 batch id 1 loss 0.09952840954065323 train acc 0.984375
epoch 9 batch id 201 loss 0.030905332416296005 train acc 0.9878731343283582
epoch 9 batch id 401 loss 0.0031795362010598183 train acc 0.9860504987531172
epoch 9 batch id 601 loss 0.05599892511963844 train acc 0.9859608985024958
epoch 9 batch id 801 loss 0.00523745222017169 train acc 0.9860720973782772
epoch 9 batch id 1001 loss 0.030446259304881096 train acc 0.9861232517482518
epoch 9 batch id 1201 loss 0.02420222945511341 train acc 0.986417568692756
epoch 9 batch id 1401 loss 0.09632450342178345 train acc 0.9865497858672377
epoch 9 batch id 1601 loss 0.002402503741905093 train acc 0.9863464241099313
epoch 9 batch id 1801 loss 0.1333174854516983 train acc 0.9862055802332038
epoch 9 batch id 2001 loss 0.003359818132594228 train acc 0.9863037231384307
epoch 9 batch id 2201 loss 0.011002270504832268 train acc 0.9863201385733757
epoch 9 batch id 2401 loss 0.07389777898788452 train acc 0.9862492190753852
epoch 9 batch id 2601 lo


epoch 9 test acc 0.7756193693693694 test loss 1.9291362762451172


epoch 10 batch id 1 loss 0.11138397455215454 train acc 0.984375
epoch 10 batch id 201 loss 0.010439303703606129 train acc 0.9881840796019901


KeyboardInterrupt: ignored

# Inference

In [ ]:
#새로운 문장 테스트
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("불안이")
            elif np.argmax(logits) == 1:
                test_eval.append("당황이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

using cached model


In [53]:
checkpoint=torch.load(ckpt_name)

In [54]:
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 짜증나네


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 열받게 하네
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 죽고싶어
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 어떻게 하면 좋지
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 걱정이 태산이야
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 어떻게 해야할지 모르겠어
>> 입력하신 내용에서 불안이 느껴집니다.


하고싶은 말을 입력해주세요 : 남친이랑 싸웠어
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 여친이랑 싸웠어
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 행복해
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 우울하지 않아
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 즐겁지 않아
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 행복하지 않아
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 안우울해
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 안즐거워
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 안슬퍼
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 남자친구랑 헤어졌어
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 역겨워
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 그냥 그래
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 뭐?
>> 입력하신 내용에서 당황이 느껴집니다.


하고싶은 말을 입력해주세요 : 나보고 어쩌라고.....
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 

KeyboardInterrupt: ignored